<a href="https://colab.research.google.com/github/IsharaSilva/Prediction-Recommedation-system-for-Potato-Cultivation/blob/main/Module_2_Detection_of_Insect_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rembg

In [ ]:
import numpy as np # dataAnalysis
import pandas as pd #multidimensional arrays
import matplotlib.pyplot as plt #data visualization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import img_as_ubyte
from skimage.util import img_as_float
from rembg import remove

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_folder_path = 'https://drive.google.com/drive/folders/1J_XyVRFE15SzLjFtT4Z1OrabYlNeDHGQ?usp=share_link'

In [ ]:
!git clone https://github.com/IsharaSilva/Prediction-Recommedation-system-for-Potato-Cultivation.git

In [ ]:
import os
path = os.listdir('/content/drive/MyDrive/Insect/Training')
classes = {'Insect_Coloradopotatobeetle':0, 'Insect_Fleabeetle':1}

In [ ]:
import cv2
import os

img_no = 1
x = []
y = []
z = []
histo = []

for cls in classes:
    input_path = '/content/drive/MyDrive/Insect/Training/' + cls
    output_path = '/content/drive/MyDrive/Insect/Preprocessed/' + cls

    # Create output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for j in os.listdir(input_path):
        realimage = cv2.imread(os.path.join(input_path, j), 1)

        # Preprocessing steps
        resizedImage = cv2.resize(realimage, (400, 400))

        color_contrastImage = img_as_ubyte(cv2.cvtColor(resizedImage, cv2.COLOR_BGR2RGB))

        smooth_image = img_as_ubyte(cv2.GaussianBlur(color_contrastImage, (5, 5), 0, borderType=cv2.BORDER_CONSTANT))

        R, G, B = cv2.split(smooth_image)
        op_R = cv2.equalizeHist(R)
        op_G = cv2.equalizeHist(G)
        op_B = cv2.equalizeHist(B)
        histogram_image = cv2.merge((op_R, op_G, op_B))

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        op2_R = clahe.apply(R)
        op2_G = clahe.apply(G)
        op2_B = clahe.apply(B)
        clahe_image = cv2.merge((op2_R, op2_G, op2_B))

        # Save the preprocessed image to the output folder
        #output_filename = os.path.join(output_path, str(img_no) + '.jpg')
        #cv2.imwrite(output_filename, clahe_image)

        x.append(clahe_image)
        y.append(classes[cls])
        z.append(realimage)
        histo.append(histogram_image)

        img_no += 1

In [ ]:
plt.imshow(x[150])

In [ ]:
plt.imshow(z[150])

In [ ]:
classes = ['Insect_Coloradopotatobeetle', 'Insect_Fleabeetle']

for cls in classes:
    input_path = f'/content/drive/MyDrive/Insect/Preprocessed/'+ cls
    output_path = f'/content/drive/MyDrive/Insect/AugmentationImages/'+ cls

    target_size = 400

    # Augmentation parameters
    rotation_range = 30  # Rotation angle range in degrees
    scale_range = 0.2  # Scaling factor range

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Get the list of image files in the dataset directory
    image_files = [file for file in os.listdir(input_path) if file.endswith(".jpg") or file.endswith(".png")]

    # Shuffle the image files list
    np.random.shuffle(image_files)

    # Loop over each image file
    for file in image_files:
        if len(image_files) >= target_size:
            break

        # Read the image
        image_path = os.path.join(input_path, file)
        print(f"Reading image: {image_path}")
        image = cv2.imread(image_path)

        if image is None:
            print(f"Failed to read image: {image_path}")
            continue

        # Apply random augmentation
        for i in range(10):  # Generate 10 augmented versions for each image
            # Apply random rotation
            angle = np.random.uniform(-rotation_range, rotation_range)
            rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
            rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

            # Apply random flipping
            flip_direction = np.random.randint(-1, 2)
            flipped_image = cv2.flip(rotated_image, flip_direction)

            # Save augmented image
            augmented_file = f"augmented_{len(image_files)}.jpg"
            augmented_path = os.path.join(output_path, augmented_file)
            #cv2.imwrite(augmented_path, transformed_image)

            # Add augmented image to the list
            image_files.append(augmented_file)

            if len(image_files) >= target_size:
                break

        if len(image_files) % 50 == 0:
            print(f"Generated {len(image_files)} augmented images.")

print("Data augmentation complete!")

In [ ]:
classes = ['Insect_Coloradopotatobeetle', 'Insect_Fleabeetle']
output_base_path = '/content/drive/MyDrive/Insect/bgremoveImagesNew/'

img_no = 1  # Counter for saving the processed images

for cls in classes:
    input_path = f'/content/drive/MyDrive/Insect/preprocessing_augmentation/{cls}'

    # Create the output directory for the class if it doesn't exist
    output_path = os.path.join(output_base_path, cls)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for j, file in enumerate(os.listdir(input_path)):
        image_path = os.path.join(input_path, file)
        realimage = cv2.imread(image_path, 1)
        img1 = cv2.resize(realimage, (400, 400))
        new_img = img_as_ubyte(img1)
        bg_rem_img = remove(new_img)
        bg_rem_img_BGR = cv2.cvtColor(bg_rem_img, cv2.COLOR_BGR2RGB)
        x.append(bg_rem_img)

        # Save the processed image in the class-specific output directory
        output_file = os.path.join(output_path, f'{img_no}.jpg')
        cv2.imwrite(output_file, bg_rem_img_BGR)

        img_no += 1

print("Processing and saving complete!")

In [ ]:
import matplotlib.pyplot as plt

# Path to the directory containing the processed images
processed_images_path = '/content/drive/MyDrive/Insect/bgremoveImagesNew'

# Function to display a specific image
def show_image(cls, img_num):
    class_output_path = os.path.join(processed_images_path, cls)
    image_path = os.path.join(class_output_path, f'{img_num}.jpg')
    img = plt.imread(image_path)
    plt.imshow(img)
    plt.title(f'Class: {cls} - Image: {img_num}')
    plt.axis('off')
    plt.show()

# Choose the image to display
cls = 'Insect_Coloradopotatobeetle'
img_num = 201  # Specify the image number

# Display the chosen image
show_image(cls, img_num)


In [ ]:
# path = os.listdir('/content/drive/MyDrive/Insect/bgremoveImagesNew/')
# classes = {'Insect_Coloradopotatobeetle':0, 'Insect_Fleabeetle':1}

path = os.listdir('/content/drive/MyDrive/Insect/bgRemove/')
classes = {'Colorado':0, 'flea':1}

In [ ]:
# x=[]
# y=[]
# z=[]

# save_folder = '/content/drive/MyDrive/Insect/Segmentation/'
# for cls in classes:
#     pth = '/content/drive/MyDrive/Insect/bgremoveImagesNew/'+cls
#     for j in os.listdir(pth):
#         img = cv2.imread(pth+'/'+j,1) 

#         #Canny Edge Detection,identify the boundaries (edges) of objects,
#         edges = cv2.Canny(image=img, threshold1=10, threshold2=20)
#         edgedetect_image=cv2.cvtColor(edges, cv2.COLOR_BGR2RGB)

#         # Thresholding segmentation method
#         gray = cv2.cvtColor(edgedetect_image, cv2.COLOR_BGR2GRAY)
#         _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

#         # Create a green mask with the same size as the image
#         mask = np.full_like(thresh, 255, dtype=np.uint8)

#         # Apply the mask to the image
#         result = cv2.bitwise_and(thresh, mask)
        
#         x.append(result)
#         y.append(classes[cls])
#         z.append(img)
 
  #Save the image in the specified folder
        #filename = os.path.join(save_folder, j)
        #cv2.imwrite(filename, result) 

################################  
x=[]
y=[]
z=[]

save_folder = '/content/drive/MyDrive/Insect/Segmentation/'
for cls in classes:
    pth = '/content/drive/MyDrive/Insect/bgRemove/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j,1) 

        #Canny Edge Detection,identify the boundaries (edges) of objects,
        edges = cv2.Canny(image=img, threshold1=10, threshold2=20)
        edgedetect_image=cv2.cvtColor(edges, cv2.COLOR_BGR2RGB)

        # Thresholding segmentation method
        gray = cv2.cvtColor(edgedetect_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Create a green mask with the same size as the image
        mask = np.full_like(thresh, 255, dtype=np.uint8)

        # Apply the mask to the image
        result = cv2.bitwise_and(thresh, mask)
        
        x.append(result)
        y.append(classes[cls])
        z.append(img)
 
#   # Save the image in the specified folder
#         #filename = os.path.join(save_folder, j)
#         #cv2.imwrite(filename, result) 

In [ ]:
plt.imshow(x[180])

In [ ]:
classes = {'Insect_Coloradopotatobeetle':0, 'Insect_Fleabeetle':1}
class_counts = {}

for cls in classes:
    pth = '/content/drive/MyDrive/Insect/bgremoveImagesNew/' + cls
    file_count = len([f for f in os.listdir(pth) if os.path.isfile(os.path.join(pth, f))])
    class_counts[cls] = file_count

# Print the number of images for each class
for cls, count in class_counts.items():
    print(f"Number of images for class {cls}: {count}")

In [ ]:
np.unique(y)

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
#Prepare data
x_updated = x.reshape(len(x), -1)
x_updated.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into train and test subsets
xtrain, xtest,ytrain,ytest = train_test_split(x_updated, y, test_size=0.2, random_state=30)

# Further split the test subset into validation and final test subsets
xval, xtest, yval, ytest = train_test_split(xtest, ytest, test_size=0.2, random_state=30)

# Print the sizes of each subset
# print("Training set size:", len(xtrain))
# print("Validation set size:", len(xval))
# print("Testing set size:", len(xtest))

In [ ]:
xtrain.shape,xtest.shape

In [ ]:
#feature extraction and dimensionality reduction
from sklearn.decomposition import PCA

import joblib
#Feature Selection : PCA
print(xtrain.shape, xtest.shape)
pca = PCA(n_components=0.98)
#pca_train = pca.fit_transform(X_train_std)
#pca_test = pca.transform(X_test_std)
pca_train=xtrain
pca_test=xtest

# Save the PCA object
joblib.dump(pca, '/content/drive/MyDrive/Insect/Insect Detection Model/new_pca1.pkl')

In [ ]:
from sklearn.svm import SVC

sv = SVC()
sv.fit(pca_train, ytrain)

print("Training Score:",sv.score(pca_train,ytrain))
print("Classification Score:",sv.score(pca_test,ytest))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
knn.fit(pca_train, ytrain)

# Evaluate the model
print("K-Nearest Neighbors - Training Score:", knn.score(pca_train, ytrain))
print("K-Nearest Neighbors - Classification Score:", knn.score(pca_test, ytest))


In [ ]:
import joblib

# Save the trained model to Google Drive
joblib.dump(sv, '/content/drive/MyDrive/Insect/Insect Detection Model/newsvm_model.pkl')

In [ ]:
pred=sv.predict(pca_test)
np.where(ytest!=pred)

In [ ]:
pred[8]

In [ ]:
ytest[8]

In [ ]:
# Load the trained model
model = joblib.load('/content/drive/MyDrive/Insect/Insect Detection Model/newsvm_model.pkl')

# Define the class names
class_names = ['Flea', 'Colorado']

plt.figure(figsize=(12, 8))
c = 1

for i in os.listdir('/content/drive/MyDrive/Insect/Testing/Colorado/')[:3]:
    plt.subplot(3, 3, c)
    
    img = cv2.imread('/content/drive/MyDrive/Insect/Testing/Colorado/' + i, 0)
    img1 = cv2.resize(img, (400, 400))
    img2 = img1.reshape(1, -1) / 255
    
    # Make a prediction on the image
    pred = model.predict(img2)
    pred_class_name = class_names[pred[0]]  # Use the same indexing for both classes
    
    plt.imshow(img)
    plt.axis('off')
    plt.title('Predicted: {}'.format(pred_class_name))
    
    c += 1

plt.show()

# Testing for Flea class
plt.figure(figsize=(12, 8))
c = 1

for i in os.listdir('/content/drive/MyDrive/Insect/Testing/Flea/')[:3]:
    plt.subplot(3, 3, c)
    
    img = cv2.imread('/content/drive/MyDrive/Insect/Testing/Flea/' + i, 0)
    img1 = cv2.resize(img, (400, 400))
    img2 = img1.reshape(1, -1) / 255

    # Define the class names
    class_names = ['Colorado', 'Flea']
    
    # Make a prediction on the image
    pred = model.predict(img2)
    pred_class_name = class_names[pred[0]]  # Use the same indexing for both classes
    
    plt.imshow(img)
    plt.axis('off')
    plt.title('Predicted: {}'.format(pred_class_name))
    
    c += 1

plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision, recall, and F1 score on the training and testing data
train_precision = precision_score(ytrain, sv.predict(pca_train), average='weighted')
test_precision = precision_score(ytest, sv.predict(pca_test), average='weighted')

train_recall = recall_score(ytrain, sv.predict(pca_train), average='weighted')
test_recall = recall_score(ytest, sv.predict(pca_test), average='weighted')

train_f1 = f1_score(ytrain, sv.predict(pca_train), average='weighted')
test_f1 = f1_score(ytest, sv.predict(pca_test), average='weighted')

print("Training Precision:", train_precision)
print("Testing Precision:", test_precision)

print("Training Recall:", train_recall)
print("Testing Recall:", test_recall)

print("Training F1 Score:", train_f1)
print("Testing F1 Score:", test_f1)

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report on the testing data
classification_report_test = classification_report(ytest, sv.predict(pca_test), target_names=class_names)

print("Classification Report (Testing Data):")
print(classification_report_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Calculate and visualize the confusion matrix on the testing data
cm = confusion_matrix(ytest, sv.predict(pca_test))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy score on the training and testing data
train_accuracy = accuracy_score(ytrain, sv.predict(pca_train))
test_accuracy = accuracy_score(ytest, sv.predict(pca_test))

# Print the accuracy scores
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)